In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahack-machine-learning-for-banking/test_fjtUOL8.csv
/kaggle/input/janatahack-machine-learning-for-banking/sample_submission_HSqiq1Q.csv
/kaggle/input/janatahack-machine-learning-for-banking/train_fNxu4vz.csv


In [3]:
train=pd.read_csv("/kaggle/input/janatahack-machine-learning-for-banking/train_fNxu4vz.csv")
test=pd.read_csv("/kaggle/input/janatahack-machine-learning-for-banking/test_fjtUOL8.csv")
sample=pd.read_csv("/kaggle/input/janatahack-machine-learning-for-banking/sample_submission_HSqiq1Q.csv")

train.shape,test.shape,sample.shape

((164309, 14), (109541, 13), (109541, 2))

In [4]:
train['Loan_Amount_Requested']=train['Loan_Amount_Requested'].apply(lambda x:''.join(e for e in x if e.isdigit() or e == '.'))
test['Loan_Amount_Requested']=test['Loan_Amount_Requested'].apply(lambda x:''.join(e for e in x if e.isdigit() or e == '.'))

train['Loan_Amount_Requested']=train['Loan_Amount_Requested'].astype(int)
test['Loan_Amount_Requested']=test['Loan_Amount_Requested'].astype(int)

In [ ]:
test.isnull().sum()

In [ ]:
train.dtypes

In [ ]:
train['Length_Employed'].unique()

In [ ]:
df=train.loc[train['Length_Employed']=="< 1 year"]
df['Home_Owner'].value_counts()

In [5]:
train1=train.fillna(-999,axis=1)
test1=test.fillna(-999,axis=1)
train1.isnull().sum()

Loan_ID                    0
Loan_Amount_Requested      0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Gender                     0
Interest_Rate              0
dtype: int64

In [6]:
train1['Gender']=train1['Gender'].map({"Female":0,"Male":1})
test1['Gender']=test1['Gender'].map({"Female":0,"Male":1})

train1['Home_Owner']=train1['Home_Owner'].map({"None":0,"Other":1,"Mortgage":2,"Rent":3,"Own":4,-999:-999})
test1['Home_Owner']=test1['Home_Owner'].map({"None":0,"Other":1,"Mortgage":2,"Rent":3,"Own":4,-999:-999})


train1['Length_Employed']=train1['Length_Employed'].map({'< 1 year':0,'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'10+ years':10,-999:-999})
test1['Length_Employed']=test1['Length_Employed'].map({'< 1 year':0,'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'10+ years':10,-999:-999})

train1['Income_Verified']=train1['Income_Verified'].map({'not verified':0,'VERIFIED - income':1,'VERIFIED - income source':2})
test1['Income_Verified']=test1['Income_Verified'].map({'not verified':0,'VERIFIED - income':1,'VERIFIED - income source':2})

train1['Purpose_Of_Loan']=train1['Purpose_Of_Loan'].map(lambda x: 6 if x=="debt_consolidation" else 5 if x=="credit_card" else 4 if x=="home_improvement" else 3 if x=="other" else 2 if x=="major_purchase" else 1)
test1['Purpose_Of_Loan']=test1['Purpose_Of_Loan'].map(lambda x: 6 if x=="debt_consolidation" else 5 if x=="credit_card" else 4 if x=="home_improvement" else 3 if x=="other" else 2 if x=="major_purchase" else 1)

In [7]:
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier
cat_features1=['Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Gender']
test2=test1.drop('Loan_ID',axis=1)
train2=train1.drop(columns=['Loan_ID','Interest_Rate'],axis=1)
y=train1['Interest_Rate']

In [ ]:

SEED = 1
params = {'iterations':2000, #2000->best settings
          'learning_rate':0.1, #best->.1
    'random_strength':0.1,
    'depth':4, #best 4
    #'l2_leaf_reg' : 14,
    'loss_function':'MultiClass', # objective function
          'eval_metric':'Accuracy', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': SEED,'od_type':'Iter',
                            
                            'od_wait':50,
           
         }


cbc_1 = CatBoostClassifier(**params)

In [ ]:
from sklearn.model_selection import KFold
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini
K = 5 #5->best
kf = KFold(n_splits = K, random_state = 1, shuffle = True)

OPTIMIZE_ROUNDS = False
y = y
X = train2
y_valid_pred = 0*y
X_test = test2

y_test_pred1= 0
y_test_pred2= 0
y_test_pred3= 0

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X,y)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
    print( "\nFold ", i)
    
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        fit_model = cbc_1.fit( X_train, y_train, 
                               eval_set=[X_valid, y_valid],
                               use_best_model=True#,cat_features=cat_features1
                             )
        print( "  N trees = ", model.tree_count_ )
    else:
        fit_model = cbc_1.fit( X_train, y_train)# ,cat_features=cat_features1)
        
    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    y_test_pred1 += fit_model.predict_proba(X_test)[:,0]
    y_test_pred2 += fit_model.predict_proba(X_test)[:,1]
    y_test_pred3 += fit_model.predict_proba(X_test)[:,2]
y_test_pred1 /= K  # Average test set predictions
y_test_pred2 /= K 
y_test_pred3 /= K 

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)

In [ ]:
pred_df_cat=pd.DataFrame({1:y_test_pred1,2:y_test_pred2,3:y_test_pred3})
pred_df_cat.head()

In [ ]:
pred_df_cat['class']=pred_df_cat.apply(lambda x: x.argmax(), axis=1)

In [ ]:
pred_df_cat['class1']=pred_df_cat['class'].apply(lambda x:x+1)
pred_df_cat['class1'].unique()

In [ ]:
pred_df_cat['class1'].head()

In [ ]:
sample['Interest_Rate']=pred_df_cat['class1']

In [ ]:
sample.to_csv("/kaggle/working/submit14_catboost.csv",index=False)

In [ ]:
import lightgbm as lgb
print ('Training lightgbm')

params = {
    'n_estimators':2000,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': ['multi_error'],
    "learning_rate": 0.1, #.05
     "num_leaves": 60,
     "max_depth": 5,
    'min_child_samples': 100,  
    
    'reg_alpha': 7,  
    'reg_lambda': 7,  
   # 'nthread': 8,
    'verbose': 0,
    'num_class':3,
    #'early_stopping_round':100
     
    }
lgb1 = lgb.LGBMClassifier(**params)

In [ ]:
from sklearn.model_selection import KFold
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini


OPTIMIZE_ROUNDS = False
y = y
X = train2
y_valid_pred = 0*y
X_test = test2


y_test_pred1= 0
y_test_pred2= 0
y_test_pred3= 0
feats=list(train2.columns)


In [ ]:
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)

OPTIMIZE_ROUNDS = False

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
feature_importance_df = pd.DataFrame()
for i, (train_index, test_index) in enumerate(kf.split(X,y)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
    print( "\nFold ", i)
    
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        fit_model = lgb1.fit( X_train, y_train, 
                               eval_set=[X_valid, y_valid],
                               use_best_model=True
                             )
        print( "  N trees = ", model.tree_count_ )
    else:
        fit_model = lgb1.fit( X_train, y_train)


 
    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    #print('Fold %2d AUC : %.6f' % (i + 1, roc_auc_score(y_valid, pred)))
    
   
    
    
    # Accumulate test set predictions
    y_test_pred1 += fit_model.predict_proba(X_test)[:,0]
    y_test_pred2 += fit_model.predict_proba(X_test)[:,1]
    y_test_pred3 += fit_model.predict_proba(X_test)[:,2]
y_test_pred1 /= K  # Average test set predictions
y_test_pred2 /= K 
y_test_pred3 /= K
print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


In [ ]:
pred_df_lgb=pd.DataFrame({1:y_test_pred1,2:y_test_pred2,3:y_test_pred3})
pred_df_lgb.head()

In [ ]:
pred_df_lgb['class']=pred_df_lgb.apply(lambda x: x.argmax(), axis=1)


In [ ]:
pred_df_lgb['class1']=pred_df_lgb['class'].apply(lambda x:x+1)
pred_df_lgb['class1'].unique()


In [ ]:
sample['Interest_Rate']=pred_df_lgb['class1']
sample.head()

In [ ]:
sample.to_csv("/kaggle/working/submit4_lgb.csv",index=False)

In [9]:

from sklearn.model_selection import StratifiedKFold
import xgboost as xgb

In [11]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [12]:
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [94]:
params = {
    'min_child_weight': 10.0,
    'objective': 'multi:softprob',
    'max_depth': 7,  #best:5
    'num_class':3,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.3, #best ->0.3
    'gamma': 0.65,
    'num_boost_round' : 700
    }

In [79]:
train['Interest_Rate1']=train['Interest_Rate'].map({1:0,2:1,3:2})
train['Interest_Rate1'].unique()

array([0, 2, 1])

In [95]:
X = train2.values
y = train['Interest_Rate1'].values
#test_id = test.Loan_ID.values
test = test2
xgb_preds = []

In [96]:
for train_index, test_index in skf.split(X,y):
    train_X, valid_X = X[train_index], X[test_index]
    train_y, valid_y = y[train_index], y[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = params

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test.values)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=50, early_stopping_rounds=100)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

[17:49:38] WARNING: /workspace/src/learner.cc:328: 
Parameters: { num_boost_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.50911	valid-merror:0.51421	train-gini:-0.31889	valid-gini:-0.30803
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[50]	train-merror:0.41592	valid-merror:0.46181	train-gini:-0.53268	valid-gini:-0.48670
[100]	train-merror:0.39667	valid-merror:0.46285	train-gini:-0.55444	valid-gini:-0.48975
Stopping. Best iteration:
[0]	train-merror:0.50911	valid-merror:0.51421	train-gini:-0.31889	valid-gini:-0.30803

[17:50:08] WARNING: /workspace/src/learner.cc:328: 
Parameters: { num_boost_round } might not be used.

  This may not be accurate due to

In [97]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(kfold):
        sum+=xgb_preds[j][i]
    preds.append(sum / kfold)

In [98]:
y_test_pred1=[]
for j in range(0,len(preds)):
    #print(j)
    y_test_pred1.append(preds[j][0])

In [99]:
y_test_pred2=[]
for j in range(0,len(preds)):
    #print(j)
    y_test_pred2.append(preds[j][1])

In [100]:
y_test_pred3=[]
for j in range(0,len(preds)):
    #print(j)
    y_test_pred3.append(preds[j][2])

In [101]:
pred_df_xgb=pd.DataFrame({1:y_test_pred1,2:y_test_pred2,3:y_test_pred3})
pred_df_xgb.head()

,1,2,3
0,0.177429,0.368925,0.453646
1,0.498442,0.411814,0.089744
2,0.149787,0.382555,0.467658
3,0.184730,0.530925,0.284346
4,0.098397,0.563988,0.337615


In [ ]:
	1	2	3
0	0.182290	0.371077	0.446633
1	0.523346	0.395524	0.081130
2	0.187881	0.403601	0.408519
3	0.209714	0.518210	0.272076
4	0.137816	0.524658	0.337526

In [102]:
pred_df_xgb['class']=pred_df_xgb.apply(lambda x: x.argmax(), axis=1)

In [ ]:
pred_df_xgb.shape

In [104]:
pred_df_xgb['class1']=pred_df_xgb['class'].apply(lambda x:x+1)
pred_df_xgb['class1'].unique()

array([3, 1, 2])

In [105]:
sample['Interest_Rate']=pred_df_xgb['class1']
sample.head()

,Loan_ID,Interest_Rate
0,10164310,3
1,10164311,1
2,10164312,3
3,10164313,2
4,10164314,2


In [93]:
sample.to_csv("/kaggle/working/submit9_xgb.csv",index=False)

In [ ]:
ensemble_preds_70_30_00 = 0.7 * pred_df_xgb + 0.3 * pred_df_cat + 0.0 * pred_df_lgb 
ensemble_preds_70_25_05 = 0.7 * pred_df_xgb + 0.25 * pred_df_cat + 0.05 * pred_df_lgb

In [ ]:
ensemble_preds_70_30_00

In [ ]:
ensemble_preds_70_30_00.drop(columns=["class","class1"],axis=1,inplace=True)
ensemble_preds_70_30_00.head()

In [ ]:

ensemble_preds_70_25_05.drop(columns=["class","class1"],axis=1,inplace=True)
ensemble_preds_70_25_05.head()

In [ ]:
ensemble_preds_70_30_00['class']=ensemble_preds_70_30_00.apply(lambda x: x.argmax(), axis=1)

In [ ]:
ensemble_preds_70_25_05['class']=ensemble_preds_70_25_05.apply(lambda x: x.argmax(), axis=1)

In [ ]:
ensemble_preds_70_25_05['class1']=ensemble_preds_70_25_05['class'].apply(lambda x:x+1)
ensemble_preds_70_25_05['class1'].unique()

In [ ]:
ensemble_preds_70_30_00['class1']=ensemble_preds_70_30_00['class'].apply(lambda x:x+1)
ensemble_preds_70_30_00['class1'].unique()

In [ ]:
sample['Interest_Rate']=ensemble_preds_70_30_00['class1']
sample.head()

In [ ]:
sample['Interest_Rate']=ensemble_preds_70_25_05['class1']
sample.head()

In [ ]:
sample.to_csv("/kaggle/working/ensemble1_preds_70_25_05.csv",index=False)

In [ ]:
sample.to_csv("/kaggle/working/ensemble1_preds_70_30_00.csv",index=False)

In [ ]:
percent_missing = train.isnull().sum() * 100 / len(train)
missing_value_df = pd.DataFrame({'column_name': train.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
train['Loan_Amount_Requested']=train['Loan_Amount_Requested'].apply(lambda x:''.join(e for e in x if e.isdigit() or e == '.'))
test['Loan_Amount_Requested']=test['Loan_Amount_Requested'].apply(lambda x:''.join(e for e in x if e.isdigit() or e == '.'))

In [ ]:
train['Loan_Amount_Requested']=train['Loan_Amount_Requested'].astype(int)
test['Loan_Amount_Requested']=test['Loan_Amount_Requested'].astype(int)

In [ ]:
train.hist(figsize = (15,20))

In [ ]:
test.hist(figsize = (15,20))

In [ ]:
import seaborn as sns
ax = sns.countplot(x="Interest_Rate", data=train)

In [ ]:
ax = sns.countplot(x="Interest_Rate", hue="Gender", data=train)

In [ ]:
g = sns.catplot(x="Home_Owner", y="Loan_Amount_Requested", hue="Interest_Rate", data=train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,25))
g = sns.catplot(x="Length_Employed", y="Annual_Income", hue="Interest_Rate", data=train)

In [ ]:
g = sns.catplot(x="Home_Owner", y="Loan_Amount_Requested", hue="Interest_Rate",
               data=train, kind="violin")

In [ ]:
g = sns.catplot("Interest_Rate", col="Length_Employed", col_wrap=3,
                data=train,
                kind="count", height=2.5, aspect=1.5)

In [ ]:

g = sns.catplot("Interest_Rate", col="Purpose_Of_Loan", col_wrap=4,
                data=train,
                kind="count", height=2.5, aspect=1.5)

In [ ]:

g = sns.catplot("Interest_Rate", col="Home_Owner", col_wrap=4,
                data=train,
                kind="count", height=2.5, aspect=1.5)

In [ ]:
g = sns.catplot(x="Length_Employed", col="Interest_Rate",
                data=train, kind="count",
                height=3, aspect=2);

In [ ]:
g = sns.catplot(x="Interest_Rate", y="Loan_Amount_Requested", 
                data=train, saturation=.5,
                kind="bar", ci=None, aspect=.6)

In [ ]:
g = sns.catplot(x="Interest_Rate", y="Debt_To_Income", 
                data=train, saturation=.5,
                kind="bar", ci=None, aspect=.6)

In [ ]:

g = sns.catplot(x="Interest_Rate", y="Months_Since_Deliquency", 
                data=train, saturation=.5,
                kind="bar", ci=None, aspect=.6)